In [25]:
import pandas as pd

df = pd.read_parquet("sih_rs_tratado.parquet")

In [42]:
import polars as pl
from pathlib import Path

# --- Configuração ---
# Ajuste o caminho se necessário
PROCESSED_FILE_PATH = Path("sih_rs_tratado.parquet")

def analisar_frequencia_etnia(caminho_arquivo: Path):
    """
    Lê um arquivo Parquet e conta a frequência dos 10 valores mais comuns
    na coluna 'ETNIA'.
    """
    if not caminho_arquivo.exists():
        print(f"ERRO: Arquivo não encontrado em '{caminho_arquivo}'")
        return

    print(f"Analisando o arquivo: {caminho_arquivo.name}")
    print("-" * 40)

    try:
        # 1. Lê o arquivo, selecionando apenas a coluna 'ETNIA' para economizar memória
        df = pl.read_parquet(caminho_arquivo, columns=["DIAG_SECUN"])

        # 2. Conta a frequência de cada valor na coluna, ordena do mais frequente
        #    para o menos, e pega os 10 primeiros.
        contagem_etnia = (
            df.group_by("DIAG_SECUN")
            .agg(pl.count().alias("contagem")) # Conta as ocorrências e renomeia para 'contagem'
            .sort("contagem", descending=True) # Ordena pela contagem
            .limit(10)                         # Pega os 10 primeiros
        )

        print("Contagem dos 10 valores mais frequentes para a coluna 'ETNIA':")
        print(contagem_etnia)

    except pl.ColumnNotFoundError:
        print("ERRO: A coluna 'ETNIA' não foi encontrada no arquivo.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

# --- Execução ---
if __name__ == "__main__":
    analisar_frequencia_etnia(PROCESSED_FILE_PATH)



Analisando o arquivo: sih_rs_tratado.parquet
----------------------------------------


/tmp/ipykernel_17885/3903583166.py:28: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("contagem")) # Conta as ocorrências e renomeia para 'contagem'


Contagem dos 10 valores mais frequentes para a coluna 'ETNIA':
shape: (10, 2)
┌────────────┬──────────┐
│ DIAG_SECUN ┆ contagem │
│ ---        ┆ ---      │
│ str        ┆ u32      │
╞════════════╪══════════╡
│ 0          ┆ 11312728 │
│ W199       ┆ 101436   │
│ Z370       ┆ 24124    │
│ X588       ┆ 19745    │
│ O800       ┆ 17858    │
│ W999       ┆ 16744    │
│ Y349       ┆ 15646    │
│ W029       ┆ 12965    │
│ V499       ┆ 7809     │
│ J180       ┆ 7695     │
└────────────┴──────────┘


In [34]:
import pandas as pd
from pathlib import Path

# Define the file path. Assumes the notebook is in the 'interim' folder.
file_path = Path("sih_rs_tratado.parquet")

if not file_path.exists():
    print(f"Error: The file '{file_path}' was not found. Make sure the pipeline's step 3 has been run and the notebook is in the correct folder.")
else:
    try:
        # Read the entire Parquet file into a pandas DataFrame.
        # This will show all columns that exist in the file.
        df = pd.read_parquet(file_path)

        # Show all existing columns to confirm if DIAS_CALCULADOS was created.
        print("All columns found in the file:")
        print(df.columns.tolist())

        # Now, try to select the relevant columns.
        if "DIAS_CALCULADOS" in df.columns and "DIAS_PERM" in df.columns:
            df_selected = df[["N_AIH", "DT_INTER", "DT_SAIDA", "DIAS_PERM", "DIAS_CALCULADOS"]]
            
            print("\nComparing DIAS_PERM and DIAS_CALCULADOS:")
            # Show a sample of records where the values are different.
            df_diff = df_selected[df_selected["DIAS_PERM"] != df_selected["DIAS_CALCULADOS"]].head(20)
            print(df_diff)

            # Print a summary of the differences.
            total_diff = len(df_selected[df_selected["DIAS_PERM"] != df_selected["DIAS_CALCULADOS"]])
            total_records = len(df_selected)
            print(f"\nTotal records checked: {total_records:,}")
            print(f"Total records with a difference: {total_diff:,}")
        else:
            print("\nError: The required columns 'DIAS_CALCULADOS' or 'DIAS_PERM' were not found in the file. Please check your preprocess script.")

    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

All columns found in the file:
['N_AIH', 'ESPEC', 'IDENT', 'CEP', 'MUNIC_RES', 'NASC', 'SEXO', 'DT_INTER', 'DT_SAIDA', 'UTI_MES_TO', 'MARCA_UTI', 'UTI_INT_TO', 'DIAR_ACOM', 'QT_DIARIAS', 'PROC_REA', 'VAL_SH', 'VAL_SP', 'VAL_TOT', 'VAL_UTI', 'NATUREZA', 'CNES', 'NAT_JUR', 'GESTAO', 'IND_VDRL', 'IDADE', 'DIAG_PRINC', 'DIAG_SECUN', 'COBRANCA', 'MORTE', 'MUNIC_MOV', 'DIAS_PERM', 'NACIONAL', 'NUM_FILHOS', 'INSTRU', 'CID_NOTIF', 'CONTRACEP1', 'CONTRACEP2', 'GESTRICO', 'INSC_PN', 'CBOR', 'CNAER', 'VINCPREV', 'INFEHOSP', 'CID_ASSO', 'CID_MORTE', 'COMPLEX', 'RACA_COR', 'ETNIA', 'DIAGSEC1', 'DIAGSEC2', 'DIAGSEC3', 'DIAGSEC4', 'DIAGSEC5', 'DIAGSEC6', 'DIAGSEC7', 'DIAGSEC8', 'DIAGSEC9', 'CGC_HOSP']

Error: The required columns 'DIAS_CALCULADOS' or 'DIAS_PERM' were not found in the file. Please check your preprocess script.


In [36]:
import pandas as pd
from pathlib import Path

# Define o caminho do arquivo.
file_path = Path("sih_rs_tratado.parquet")

if not file_path.exists():
    print(f"Erro: Arquivo '{file_path}' não encontrado. Certifique-se de que a etapa 3 do pipeline foi executada e que o notebook está na pasta correta.")
else:
    try:
        # Lê o arquivo Parquet, selecionando as colunas necessárias
        df = pd.read_parquet(file_path, columns=["N_AIH", "DT_INTER", "DT_SAIDA", "DIAS_PERM", "QT_DIARIAS"])

        # Converte as colunas de data para o tipo datetime
        df['DT_INTER'] = pd.to_datetime(df['DT_INTER'])
        df['DT_SAIDA'] = pd.to_datetime(df['DT_SAIDA'])

        # Cria a nova coluna 'DIAS_CALCULADOS'
        df['DIAS_CALCULADOS'] = (df['DT_SAIDA'] - df['DT_INTER']).dt.days

        # Exibe os registros onde há diferença entre as colunas
        df_diff = df[df['DIAS_PERM'] != df['DIAS_CALCULADOS']]

        print("Registros onde 'DIAS_PERM' é diferente de 'DIAS_CALCULADOS':")
        print(df_diff.head(20))

        print("\nResumo das diferenças:")
        total_diferentes = len(df_diff)
        total_registros = len(df)
        
        print(f"Total de registros analisados: {total_registros:,}")
        print(f"Total de registros com diferença: {total_diferentes:,}")

    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo ou processar os dados: {e}")

Registros onde 'DIAS_PERM' é diferente de 'DIAS_CALCULADOS':
            N_AIH   DT_INTER   DT_SAIDA  DIAS_PERM  QT_DIARIAS  \
0   4311103536620 2011-06-13 2011-06-15          0           0   
1   4319101186521 2019-04-01 2019-05-03          0           0   
2   4323100525381 2023-01-23 2023-01-24          0           0   
4   4321104078060 2021-05-14 2021-05-29          0           0   
5   4323104602872 2023-05-14 2023-05-15          0           0   
6   4321100421726 2021-02-06 2021-02-18          0           0   
7   4312102764816 2012-06-11 2012-06-14          0           0   
8   4319105336436 2019-08-08 2019-08-09          0           0   
9   4312104679344 2012-07-06 2012-07-13          0           0   
10  4319102001170 2019-05-08 2019-05-10          0           0   
11  4317105627990 2017-08-18 2017-08-19          0           0   
12  4313104009005 2013-06-12 2013-06-14          0           0   
13  4321105147359 2021-07-19 2021-07-22          0           0   
15  43171066114